# Import necessary libraries

In [173]:
#importing all the necessary libraries

# system

import os

# data analysis and plotting

import pandas as pd
import numpy as np
from scipy.stats import zscore
from scipy.stats import shapiro
from random import randint
import matplotlib.pyplot as plt 
import seaborn as sns
from xgboost import plot_importance
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.utils import shuffle

# data processing and model validation

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, confusion_matrix, accuracy_score, classification_report, log_loss
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold


# classification libraries

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, Matern, RationalQuadratic
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from xgboost import XGBClassifier
import lightgbm as lgb


# Importing imputation libs. 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Hyperparameter optimization

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV

# Missing data models

from itertools import combinations
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.model_selection import RepeatedStratifiedKFold
from joblib import parallel_backend

# web stuff
import pickle

# Various parameter settings

%matplotlib inline

# To install sklearn type "pip install numpy scipy scikit-learn" to the anaconda terminal

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(12,10)})

# import sys
# !conda list Check the packages installed

# Displaying all the rows/columns in a data set (the default option is not to show them)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import the data and create the datasets needed in the analysis

In [2]:
# Importing the raw data

raw_data_goldman = pd.read_csv("datasets/Goldman.csv", header = 0, encoding= 'unicode_escape')

In [3]:
raw_data_goldman.describe()

,Element:,LHUM,RHUM,LRAD,RRAD,LFEM,RFEM,LTIB,RTIB,OSCX,Metrics:,LHML,LHEB,LHHD,LHMLD,LHAPD,RHML,RHEB,RHHD,RHMLD,RHAPD,LRML,LRMLD,LRAPD,RRML,RRMLD,RRAPD,LFML,LFBL,LFEB,LFAB,LFHD,LFMLD,LFAPD,RFML,RFBL,RFEB,RFAB,RFHD,RFMLD,RFAPD,LTML,LTPB,LTMLD,LTAPD,RTML,RTPB,RTMLD,RTAPD,BIB,LIBL,RIBL,LAcH,RAcH,Derived:,Brachial,Crural,IL UL/LL,IL LL/UL,CBR FHD,McH FHD,GRINE FHD,AVG FHD
count,0.0,1538.000000,1538.000000,1538.000000,1538.000000,1538.000000,1538.000000,1538.000000,1538.000000,1538.000000,0.0,1376.000000,1354.000000,1368.000000,1376.000000,1376.000000,1403.000000,1384.000000,1396.000000,1403.000000,1402.000000,1321.000000,1321.000000,1321.000000,1337.000000,1340.000000,1340.000000,1421.000000,1416.000000,1380.000000,1378.000000,1421.000000,1423.000000,1423.000000,1426.000000,1423.000000,1386.000000,1390.000000,1435.000000,1424.000000,1424.000000,1403.000000,1352.000000,1399.000000,1398.000000,1400.000000,1349.000000,1395.000000,1394.000000,1469.000000,1179.000000,1177.000000,1371.000000,1375.000000,0.0,1463.000000,1490.000000,1418.000000,1418.000000,1519.000000,1519.000000,1519.000000,1519.000000
mean,NaN,0.100780,0.081274,0.140442,0.126788,0.070221,0.064369,0.086476,0.086476,0.020156,NaN,303.759811,57.442770,42.741615,19.884404,19.717754,307.556379,58.194581,43.003059,20.362117,20.462511,233.068887,14.097017,11.188312,234.964099,14.493866,11.335993,427.106967,423.455508,76.067754,66.361168,43.430837,25.767442,27.171792,425.657433,421.620169,76.259019,66.280950,43.497401,25.440857,27.243588,353.078403,69.344305,21.457984,26.351710,352.419429,69.343217,21.985778,25.372317,262.395848,150.980068,151.104503,48.873508,48.923360,NaN,0.765279,0.827734,0.692225,1.445639,60.145433,57.412799,62.073215,59.877149
std,NaN,0.301135,0.273345,0.347558,0.332844,0.255602,0.245489,0.281157,0.281157,0.140580,NaN,23.025881,5.446128,4.134822,2.472297,2.345870,23.116218,5.544286,4.257656,2.573130,2.381074,18.985815,1.898293,1.321821,18.904538,1.938423,1.364567,31.509788,31.597694,6.204072,5.857932,3.972412,2.745311,2.874594,31.751905,31.843808,6.238232,5.832092,4.053790,2.663129,2.915361,28.076348,5.773576,2.376611,3.022335,28.749863,5.775864,2.468692,2.783156,18.274559,10.573547,10.753604,4.052208,4.079195,NaN,0.031805,0.027455,0.018438,0.038447,7.946894,8.914306,9.029766,8.555195
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,229.500000,36.510000,29.580000,12.010000,13.000000,234.000000,42.000000,32.240000,12.250000,13.840000,179.000000,8.740000,7.370000,180.000000,9.160000,7.880000,345.000000,309.500000,58.000000,49.990000,33.640000,17.900000,18.880000,341.500000,279.000000,58.000000,49.830000,32.970000,17.970000,18.340000,276.000000,52.000000,15.220000,19.000000,237.000000,50.000000,15.150000,18.590000,184.000000,105.000000,106.000000,38.640000,37.890000,NaN,0.677912,0.692025,0.599260,1.299290,36.092754,34.692285,39.058420,38.300225
25%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,287.375000,53.000000,39.760000,18.230000,18.030000,291.000000,54.000000,39.900000,18.690000,18.732500,219.000000,12.780000,10.230000,221.000000,13.117500,10.327500,404.500000,401.000000,72.000000,62.010000,40.610000,23.830000,25.120000,403.000000,399.000000,72.000000,61.970000,40.510000,23.535000,25.097500,333.000000,65.000000,19.780000,24.180000,332.000000,65.000000,20.245000,23.380000,251.000000,145.000000,145.000000,45.995000,46.055000,NaN,0.744615,0.809212,0.680304,1.421166,54.614902,50.992205,55.569460,53.738995
50%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,303.500000,57.500000,42.715000,19.855000,19.640000,307.000000,58.000000,43.015000,20.300000,20.460000,233.000000,14.050000,11.110000,235.000000,14.400000,11.270000,428.000000,424.000000,76.000000,66.615000,43.540000,25.730000,27.160000,426.000000,422.000000,76.000000,66.190000,43.520000,25.370000,27.135000,353.000000,69.500000,21.370000,26.235000,353.000000,69.000000,21.890000,25.2

In [4]:
# Define a new data set that contains all the measured data, i.e.,
# not derived data as well as the class label Sex for each measurement

measured_data_goldman = raw_data_goldman.loc[:,"LHML":"RAcH"]

# Fill missing data with zeroes so that we can average between left and right skeletal measurements below

measured_data_goldman = measured_data_goldman.fillna(0)


In [5]:
# Create new columns and take the average between left and right skeletal measurements

target_cols = ['HML', 'HHD', 'RML', 'FML', 'FBL','FHD', 'TML', 'FEB', 'TPB', 'HEB']


for col in target_cols:
    measured_data_goldman[col] = 0.
    
    min_left_col_value = measured_data_goldman["".join(["L",col])][measured_data_goldman["".join(["L",col])] > 0.1].min() - 1
    
    min_right_col_value = measured_data_goldman["".join(["R",col])][measured_data_goldman["".join(["R",col])] > 0.1].min() - 1
    
    measured_data_goldman.loc[(measured_data_goldman["".join(["L",col])] < 0.1) & (measured_data_goldman["".join(["R",col])] > min_right_col_value), 
        col] = measured_data_goldman["".join(["R",col])]

    measured_data_goldman.loc[(measured_data_goldman["".join(["R",col])] < 0.1) & (measured_data_goldman["".join(["L",col])] > min_left_col_value), 
       col] = measured_data_goldman["".join(["L",col])]

    measured_data_goldman.loc[(measured_data_goldman["".join(["R",col])] > min_right_col_value) & (measured_data_goldman["".join(["L",col])] > min_left_col_value), 
       col] = (measured_data_goldman["".join(["L",col])] + measured_data_goldman["".join(["R",col])])/2
    
    
    
# Create a dataset with the features we will use to build our models

model_cols = ['BIB','HML', 'HHD', 'RML', 'FML', 'FBL','FHD', 'TML', 'FEB', 'TPB', 'HEB']

model_data_goldman = measured_data_goldman.drop(columns=[col for col in measured_data_goldman if col not in model_cols])

# Add the Sex column

model_data_goldman = pd.concat([model_data_goldman.loc[:,:],raw_data_goldman.loc[:,"Sex"]],axis=1)

print(model_data_goldman.describe())

               BIB          HML          HHD          RML          FML  \
count  1538.000000  1538.000000  1538.000000  1538.000000  1538.000000   
mean    250.623862   298.197042    41.699971   225.650033   420.892230   
std      57.194643    52.024020     8.025574    46.894808    57.592097   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%     249.000000   288.312500    39.520000   218.500000   402.750000   
50%     262.000000   304.500000    42.700000   233.000000   426.375000   
75%     274.000000   320.750000    45.728750   247.000000   447.937500   
max     324.000000   379.500000    55.120000   290.250000   531.750000   

               FBL          FHD          TML          FEB          TPB  \
count  1538.000000  1538.000000  1538.000000  1538.000000  1538.000000   
mean    416.541125    42.925692   343.531632    74.052666    66.309655   
std      59.136325     6.222401    63.412299    13.827096    15.227271   
min       0.000000     0.000000     0

In [6]:
# Note that the Sex column is a string, not a value, that's why
# its not printed above. 

# But we take advandage of the fact that its a string to 
# drop the values 1? and 0?

# Get rid of 1? and 0? from sex estimation and then shuffle the dataset
# because otherwise you have 1 and 0 packed together 

model_data_goldman = pd.concat([model_data_goldman.loc[model_data_goldman['Sex']=='1'], model_data_goldman.loc[model_data_goldman['Sex']=='0']])

model_data_goldman = model_data_goldman.sample(frac=1).reset_index(drop=True)

# Now convert Sex from string to int

model_data_goldman["Sex"] = model_data_goldman["Sex"].astype(int) 

model_data_goldman.describe()

,BIB,HML,HHD,RML,FML,FBL,FHD,TML,FEB,TPB,HEB,Sex
count,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000
mean,250.544830,298.170681,41.714529,225.744928,420.889234,416.533704,42.939784,343.450524,74.060209,66.297120,56.201345,0.355366
std,57.361899,52.176096,8.047175,46.673734,57.745325,59.293408,6.237500,63.587332,13.867850,15.272226,10.868670,0.478781
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,249.000000,288.250000,39.530000,218.500000,402.562500,398.937500,40.470000,330.687500,71.250000,64.500000,53.500000,0.000000
50%,262.000000,304.500000,42.735000,233.000000,426.500000,422.625000,43.460000,352.500000,76.000000,69.000000,57.500000,0.000000
75%,274.000000,320.750000,45.762500,247.000000,447.812500,444.250000,46.277500,372.000000,80.500000,73.500000,61.562500,1.000000
max,324.000000,379.500000,55.120000,290.250000,531.750000,530.500000,57.625000,444.500000,93.250000,85.000000,74.000000,1.000000


In [7]:
# replace 0.0 back to nan to better handle the dataset within xgboost but
# also to become able to drop the NA entries easily

model_data_goldman[model_cols] = model_data_goldman[model_cols].replace(0.0, np.nan)

In [8]:
model_data_goldman.describe()

,BIB,HML,HHD,RML,FML,FBL,FHD,TML,FEB,TPB,HEB,Sex
count,1459.000000,1491.000000,1487.000000,1475.000000,1508.00000,1506.000000,1509.000000,1487.000000,1486.000000,1461.000000,1486.000000,1528.000000
mean,262.393763,305.569953,42.864694,233.856441,426.47132,422.618526,43.480444,352.920242,76.153432,69.337440,57.789808,0.355366
std,18.299914,22.965932,4.148381,18.935423,31.56893,31.525125,3.984079,28.471105,6.184101,5.743871,5.441524,0.478781
min,184.000000,234.000000,29.580000,179.500000,343.75000,329.750000,33.315000,271.000000,58.000000,50.000000,41.500000,0.000000
25%,251.000000,289.500000,39.805000,220.000000,404.00000,399.812500,40.635000,332.500000,71.500000,65.000000,54.000000,0.000000
50%,263.000000,305.000000,42.920000,234.000000,427.00000,423.000000,43.505000,353.000000,76.250000,69.500000,58.000000,0.000000
75%,274.000000,321.250000,45.810000,247.500000,448.00000,444.500000,46.305000,372.250000,80.687500,73.750000,61.750000,1.000000
max,324.000000,379.500000,55.120000,290.250000,531.75000,530.500000,57.625000,444.500000,93.250000,85.000000,74.000000,1.000000


In [9]:
no_zeroes_model_data_goldman = model_data_goldman.dropna()

In [10]:
no_zeroes_model_data_goldman.describe()

,BIB,HML,HHD,RML,FML,FBL,FHD,TML,FEB,TPB,HEB,Sex
count,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000
mean,262.400154,306.085292,42.924977,233.967358,427.142665,423.216590,43.548280,353.362058,76.298771,69.439324,57.842669,0.353303
std,18.237396,22.684177,4.075788,18.760498,31.103027,31.078794,3.938374,28.230646,6.128510,5.709825,5.386024,0.478179
min,184.000000,234.000000,31.650000,179.500000,343.750000,329.750000,33.315000,276.500000,58.000000,50.000000,41.500000,0.000000
25%,251.000000,290.000000,39.913750,220.500000,405.000000,401.062500,40.735000,333.000000,71.750000,65.000000,54.000000,0.000000
50%,263.000000,305.375000,42.945000,234.000000,427.500000,423.500000,43.565000,353.375000,76.500000,69.500000,58.000000,0.000000
75%,274.000000,321.750000,45.825000,247.500000,448.000000,444.500000,46.355000,372.250000,81.000000,74.000000,61.750000,1.000000
max,324.000000,379.500000,54.760000,290.250000,531.750000,530.500000,56.195000,444.500000,93.250000,85.000000,74.000000,1.000000


In [11]:
# Create a new data set using the knn imputer
# Here we use the 3 nearest neighbors to calculate the missing data

knn_imputer = KNNImputer(n_neighbors=3, missing_values=0.0)

sex_column = model_data_goldman['Sex']

temporary_data_set = model_data_goldman.fillna(0.).drop(["Sex"],axis=1)

cols = temporary_data_set.columns

temporary_data_set = knn_imputer.fit_transform(temporary_data_set)

temporary_data_set = pd.DataFrame(data=temporary_data_set, columns=cols)

knn_imputed_data_goldman = pd.concat([temporary_data_set,sex_column],axis=1)

In [12]:
knn_imputed_data_goldman.describe()

,BIB,HML,HHD,RML,FML,FBL,FHD,TML,FEB,TPB,HEB,Sex
count,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000
mean,262.316099,305.536409,42.857743,233.813264,426.412195,422.530705,43.476048,352.991099,76.122055,69.324498,57.782715,0.355366
std,18.134815,22.874768,4.118270,18.836392,31.490822,31.449916,3.972349,28.387212,6.159489,5.705714,5.399224,0.478781
min,184.000000,234.000000,29.580000,179.500000,343.750000,329.750000,33.315000,271.000000,58.000000,50.000000,41.500000,0.000000
25%,251.000000,289.500000,39.820000,220.250000,404.187500,400.000000,40.635000,332.500000,71.500000,65.000000,54.000000,0.000000
50%,263.000000,305.000000,42.900000,234.000000,426.750000,423.000000,43.495000,353.375000,76.250000,69.500000,58.000000,0.000000
75%,274.000000,321.250000,45.790000,247.500000,448.000000,444.500000,46.291250,372.500000,80.500000,73.750000,61.750000,1.000000
max,324.000000,379.500000,55.120000,290.250000,531.750000,530.500000,57.625000,444.500000,93.250000,85.000000,74.000000,1.000000


In [13]:
# Create a new data set using the iterative imputer

iter_imputer = IterativeImputer(max_iter = 1000, missing_values=0.0)

sex_column = model_data_goldman['Sex']

temporary_data_set = model_data_goldman.fillna(0.).drop(["Sex"],axis=1)

cols = temporary_data_set.columns

temporary_data_set = iter_imputer.fit_transform(temporary_data_set)

temporary_data_set = pd.DataFrame(data=temporary_data_set, columns=cols)

iter_imputed_data_goldman = pd.concat([temporary_data_set,sex_column],axis=1)


In [14]:
iter_imputed_data_goldman.describe()

,BIB,HML,HHD,RML,FML,FBL,FHD,TML,FEB,TPB,HEB,Sex
count,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000,1528.000000
mean,262.210761,305.512210,42.849808,233.797870,426.437825,422.559829,43.479740,352.974216,76.129804,69.308226,57.781046,0.355366
std,18.081781,22.863518,4.124843,18.879069,31.498944,31.452610,3.974301,28.378044,6.167874,5.733651,5.402456,0.478781
min,184.000000,234.000000,29.580000,179.500000,343.750000,329.750000,33.315000,271.000000,58.000000,50.000000,41.500000,0.000000
25%,251.000000,289.500000,39.813750,220.250000,404.187500,400.000000,40.635000,332.500000,71.500000,65.000000,54.000000,0.000000
50%,263.000000,305.000000,42.917500,234.000000,426.750000,423.000000,43.495000,353.250000,76.250000,69.500000,58.000000,0.000000
75%,274.000000,321.062500,45.786250,247.500000,448.000000,444.500000,46.297500,372.500000,80.562500,73.696409,61.750000,1.000000
max,324.000000,379.500000,55.120000,290.250000,531.750000,530.500000,57.625000,444.500000,93.250000,85.000000,74.000000,1.000000


# Test sets

In [15]:
# Sample the data

no_zeroes_model_data_goldman = no_zeroes_model_data_goldman.sample(frac=1).reset_index(drop=True)

knn_imputed_data_goldman = knn_imputed_data_goldman.sample(frac=1).reset_index(drop=True)

iter_imputed_data_goldman = iter_imputed_data_goldman.sample(frac=1).reset_index(drop=True)

In [16]:
dataset_list = [
    no_zeroes_model_data_goldman, 
    knn_imputed_data_goldman, 
    iter_imputed_data_goldman
]

# Classification without optimization

In [17]:
classifier_names = [
    "Logistic Regression", 
    "Decision Tree Classifier", 
    "Support Vector Machines", 
    "Gaussian Process Classifier", 
    "Gradient Boosting Classifier", 
    "Random Forest Classifier",
    "Ada Boost Classifier", 
    "Extra Trees Classifier", 
    "Gaussian Naive Bayes", 
    "KNNeighbors Classifier",
    "Linear Discriminant Analysis", 
    "Quadratic Discriminant Analysis", 
    "XGBClassifier", 
    "Light Gradient Boosting Classifier"
]


classifiers = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    SVC(),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    GradientBoostingClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    ExtraTreesClassifier(),
    GaussianNB(),
    KNeighborsClassifier(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    XGBClassifier(),
    lgb.LGBMClassifier()
]

In [18]:
dataset_scores_list = []

for dataset in dataset_list:
    scores = []
    
    X = dataset.drop('Sex', axis = 1).values
    y = dataset['Sex']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size=0.3, stratify=y)
    
    for name, clf in zip(classifier_names, classifiers):
        run_score = []
        
        for i in range(20):
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)*100
            run_score.append(score)
            
            avg_score = np.mean(run_score)
                      
        #print(run_score)    
        scores.append(avg_score)
                  
    dataset_scores_list.append(scores)
       

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [19]:
dataset_scores_list

[[80.05115089514065,
  82.55754475703324,
  79.79539641943734,
  89.00255754475705,
  86.95652173913045,
  88.37595907928389,
  86.95652173913045,
  88.26086956521739,
  84.91048593350385,
  85.67774936061377,
  90.02557544757035,
  89.51406649616368,
  87.72378516624039,
  87.97953964194376],
 [82.78867102396516,
  80.67538126361656,
  79.520697167756,
  84.31372549019606,
  86.86274509803921,
  85.30501089324619,
  85.62091503267973,
  86.23093681917211,
  84.96732026143789,
  83.66013071895424,
  87.36383442265793,
  85.40305010893243,
  86.27450980392157,
  85.18518518518519],
 [83.66013071895424,
  84.04139433551198,
  83.00653594771241,
  84.53159041394336,
  90.16339869281046,
  91.27450980392156,
  87.14596949891069,
  91.63398692810458,
  89.10675381263619,
  87.14596949891069,
  90.41394335511981,
  90.19607843137257,
  91.28540305010894,
  88.88888888888889]]

In [20]:
results = pd.DataFrame(index=classifier_names)
# results['name'] = names
results['goldman_1'] = dataset_scores_list[0]
results['goldman_2'] = dataset_scores_list[1]
results['goldman_3'] = dataset_scores_list[2]

In [21]:
results

,goldman_1,goldman_2,goldman_3
Logistic Regression,80.051151,82.788671,83.660131
Decision Tree Classifier,82.557545,80.675381,84.041394
Support Vector Machines,79.795396,79.520697,83.006536
Gaussian Process Classifier,89.002558,84.313725,84.531590
Gradient Boosting Classifier,86.956522,86.862745,90.163399
Random Forest Classifier,88.375959,85.305011,91.274510
Ada Boost Classifier,86.956522,85.620915,87.145969
Extra Trees Classifier,88.260870,86.230937,91.633987
Gaussian Naive Bayes,84.910486,84.967320,89.106754
KNNeighbors Classifier,85.677749,83.660131,87.145969


In [ ]:
# print(results.to_latex(float_format="%.2f"))

# Hyperparameter optimization

In [27]:
# X = iter_imputed_data_goldman.drop('Sex', axis = 1).values
# y = iter_imputed_data_goldman['Sex']

In [28]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size=0.3, stratify=y)

In [29]:
# Logistic regression model

model = LogisticRegression()

model.fit(X_train, y_train)

model.score(X_test, y_test)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8366013071895425

In [30]:
# Optimizing the logistic regression model

model = LogisticRegression()


parameters = {
    'C': np.logspace(-2,2,40),
    'max_iter': [1000, 1500, 2000],
    'random_state': [0,1,2,3,4,5,6,7,8,9,10,11]
}

clf  = BayesSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


BayesSearchCV(cv=10, estimator=LogisticRegression(),
              search_spaces={'C': array([0.010000, 0.012664, 0.016037, 0.020309, 0.025719, 0.032570,
       0.041246, 0.052233, 0.066147, 0.083768, 0.106082, 0.134340,
       0.170125, 0.215443, 0.272833, 0.345511, 0.437548, 0.554102,
       0.701704, 0.888624, 1.125336, 1.425103, 1.804722, 2.285464,
       2.894266, 3.665241, 4.641589, 5.878016, 7.443803, 9.426685,
       11.937766, 15.117751, 19.144820, 24.244620, 30.702906, 38.881552,
       49.238826, 62.355073, 78.965229, 100.000000]),
                             'max_iter': [1000, 1500, 2000],
                             'random_state': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                              11]})

In [31]:
clf.best_params_

OrderedDict([('C', 2.2854638641349907),
             ('max_iter', 1500),
             ('random_state', 3)])

In [32]:
model = LogisticRegression(**clf.best_params_)
model.fit(X_train, y_train)
result = model.score(X_test, y_test)*100
    
print("{:10.2f}".format(result))

     89.76


In [33]:
# Support vector machines

model = SVC()

model.fit(X_train, y_train)

model.score(X_test, y_test)


0.8300653594771242

In [34]:
# Optimizing the Support Vevtor Machine model

model = SVC()

parameters = {
    'C': np.logspace(-2,2,10),
    'kernel': ['rbf','linear']
}

clf  = RandomizedSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

RandomizedSearchCV(cv=10, estimator=SVC(),
                   param_distributions={'C': array([0.010000, 0.027826, 0.077426, 0.215443, 0.599484, 1.668101,
       4.641589, 12.915497, 35.938137, 100.000000]),
                                        'kernel': ['rbf', 'linear']})

In [35]:
clf.best_params_

{'kernel': 'linear', 'C': 4.6415888336127775}

In [36]:
model = SVC(**clf.best_params_, probability=True)
model.fit(X_train, y_train)
result = model.score(X_test, y_test)*100
    
print("{:10.2f}".format(result))

     91.07


In [37]:
# kNN classifier

model = KNeighborsClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.8714596949891068

In [38]:
# Optimizing the kNN classifier

model = KNeighborsClassifier()

parameters = {
    'n_neighbors': list(range(1,21)),
    'weights' : ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan'],
    'leaf_size': list(range(1,10))
}

clf  = BayesSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

BayesSearchCV(cv=10, estimator=KNeighborsClassifier(),
              search_spaces={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                             'metric': ['euclidean', 'manhattan'],
                             'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                             12, 13, 14, 15, 16, 17, 18, 19,
                                             20],
                             'weights': ['uniform', 'distance']})

In [39]:
clf.best_params_

OrderedDict([('leaf_size', 3),
             ('metric', 'manhattan'),
             ('n_neighbors', 13),
             ('weights', 'distance')])

In [40]:
model = KNeighborsClassifier(**clf.best_params_)
model.fit(X_train, y_train)
result = model.score(X_test, y_test)*100
    
print("{:10.2f}".format(result))

     89.54


In [41]:
# Gaussian Naive Bayes

model = GaussianNB()

model.fit(X_train, y_train)

model.score(X_test, y_test)


0.8910675381263616

In [45]:
# Optimizing the Gaussian Naive Bayes classifier

model = GaussianNB()

parameters = {
    'var_smoothing': np.logspace(3,-12, num=100)
             }

clf  = GridSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

GridSearchCV(cv=10, estimator=GaussianNB(),
             param_grid={'var_smoothing': array([1000.000000, 705.480231, 497.702356, 351.119173, 247.707636,
       174.752840, 123.284674, 86.974900, 61.359073, 43.287613, 30.538555,
       21.544347, 15.199111, 10.722672, 7.564633, 5.336699, 3.764936,
       2.656088, 1.873817, 1.321941, 0.932603, 0.657933, 0.464159,
       0.327455, 0.231013, 0.162975, 0.114976, 0.081113...
       0.000001, 0.000001, 0.000001, 0.000000, 0.000000, 0.000000,
       0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000,
       0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000,
       0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000,
       0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000,
       0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000,
       0.000000, 0.000000, 0.000000, 0.000000, 0.000000])})

In [46]:
clf.best_params_

{'var_smoothing': 7.56463327554629e-05}

In [47]:
model = GaussianNB(**clf.best_params_)
model.fit(X_train, y_train)
result = model.score(X_test, y_test)*100
    
print("{:10.2f}".format(result))

     88.89


In [166]:
# Linear Discriminant Analysis

model = LinearDiscriminantAnalysis()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.9041394335511983

In [167]:
# Optimizing the Linear Discriminant Analysis classifier

model = LinearDiscriminantAnalysis()

parameters = {
    'solver' : ['svd', 'lsqr', 'eigen']
}

clf  = BayesSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point 

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=10, estimator=LinearDiscriminantAnalysis(),
              search_spaces={'solver': ['svd', 'lsqr', 'eigen']})

In [168]:
clf.best_params_

OrderedDict([('solver', 'eigen')])

In [169]:
model = LinearDiscriminantAnalysis(**clf.best_params_)
model.fit(X_train, y_train)
result = model.score(X_test, y_test)*100
    
print("{:10.2f}".format(result))

     90.41


In [170]:
pickle.dump(model, open("lda_model_20220522.dat", "wb"))

In [52]:
# Quadratic Discriminant Analysis

model = QuadraticDiscriminantAnalysis()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.9019607843137255

In [53]:
# Optimizing the Quadratic Discriminant Analysis classifier

model = QuadraticDiscriminantAnalysis()

parameters = {
    'reg_param' : [0., 0.1, 0.2, 0.3, 0.4, 0.5]
}

clf  = GridSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

GridSearchCV(cv=10, estimator=QuadraticDiscriminantAnalysis(),
             param_grid={'reg_param': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]})

In [54]:
clf.best_params_

{'reg_param': 0.3}

In [55]:
model = QuadraticDiscriminantAnalysis(**clf.best_params_)
model.fit(X_train, y_train)
result = model.score(X_test, y_test)*100
    
print("{:10.2f}".format(result))

     90.20


In [56]:
# Decision Tree Classifier

model = DecisionTreeClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.8474945533769063

In [57]:
# Optimizing the Decision Tree Classifier

model = DecisionTreeClassifier()

parameters = {
    'criterion':['entropy','gini'],
    'max_depth':[1,2,3,4,5,6,7,15,20,30,40,120,150]
}

clf  = GridSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 15, 20, 30, 40, 120,
                                       150]})

In [58]:
clf.best_params_

{'criterion': 'gini', 'max_depth': 5}

In [59]:
model = DecisionTreeClassifier(**clf.best_params_)
model.fit(X_train, y_train)
result = model.score(X_test, y_test)*100
    
print("{:10.2f}".format(result))

     89.32


In [60]:
# Random Forest Classifier

model = RandomForestClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.9150326797385621

In [61]:
# Optimizing the Random Forest Classifier

model = RandomForestClassifier()

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

clf  = RandomizedSearchCV(model, param_distributions=random_grid, n_iter = 20, cv=10, return_train_score=False)

clf.fit(X,y)

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(), n_iter=20,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]})

In [62]:
clf.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': True}

In [63]:
model = RandomForestClassifier(**clf.best_params_)
model.fit(X_train, y_train)
result = model.score(X_test, y_test)*100
    
print(result)

91.93899782135077


In [119]:
# XGBoost Classifier

model = XGBClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.9128540305010894

In [162]:
# Optimizing the XGBoost Classifier

model = XGBClassifier()

parameters = {
    'min_child_weight': [1, 2, 3, 4, 5, 6, 25, 50, 75, 100],
    'gamma': [0.1, 0.5, 1, 1.5, 2, 5, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50],
    'n_estimators': [100, 200, 300, 500, 800, 1000, 2000]
}

clf  = RandomizedSearchCV(model, parameters, cv=10)

clf.fit(X,y)

RandomizedSearchCV(cv=10, estimator=XGBClassifier(),
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.1, 0.5, 1, 1.5, 2, 5, 10],
                                        'max_depth': [3, 4, 5, 5, 6, 7, 8, 9,
                                                      10, 20, 30, 40, 50],
                                        'min_child_weight': [1, 2, 3, 4, 5, 6,
                                                             25, 50, 75, 100],
                                        'n_estimators': [100, 200, 300, 500,
                                                         800, 1000, 2000],
                                        'subsample': [0.6, 0.8, 1.0]})

In [163]:
clf.best_params_

{'subsample': 1.0,
 'n_estimators': 800,
 'min_child_weight': 50,
 'max_depth': 9,
 'gamma': 1,
 'colsample_bytree': 1.0}

In [164]:
model = XGBClassifier(**clf.best_params_)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9150326797385621


In [165]:
pickle.dump(model, open("xgb_model_20220522.dat", "wb"))

# iter_imputed_data_goldman.columns

In [81]:
# Gaussian Process Classifier

model = GaussianProcessClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.8453159041394336

In [82]:
# Optimizing the GaussianProcessClassifier

model = GaussianProcessClassifier()

parameters = {
    'kernel' : [1*RBF(), 1*DotProduct(), 1*Matern(),  1*RationalQuadratic(), 1*WhiteKernel()]
}

clf  = RandomizedSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/cconsta1/opt/anaconda3/lib/python3.9/si

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 664, in fit
    self.base_estimator_.fit(X, y)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 212, in fit
    optima = [self._constrained_optimization(obj_func,
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 445, in _constrained_optimization
    opt_res = scipy.optimize.minimize(
  File "/Users/cconsta1/opt/anaconda3/lib/p

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 664, in fit
    self.base_estimator_.fit(X, y)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 212, in fit
    optima = [self._constrained_optimization(obj_func,
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 445, in _constrained_optimization
    opt_res = scipy.optimize.minimize(
  File "/Users/cconsta1/opt/anaconda3/lib/p

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 664, in fit
    self.base_estimator_.fit(X, y)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 212, in fit
    optima = [self._constrained_optimization(obj_func,
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/_gpc.py", line 445, in _constrained_optimization
    opt_res = scipy.optimize.minimize(
  File "/Users/cconsta1/opt/anaconda3/lib/p

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better v

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better valu

RandomizedSearchCV(cv=10, estimator=GaussianProcessClassifier(),
                   param_distributions={'kernel': [1**2 * RBF(length_scale=1),
                                                   1**2 * DotProduct(sigma_0=1),
                                                   1**2 * Matern(length_scale=1, nu=1.5),
                                                   1**2 * RationalQuadratic(alpha=1, length_scale=1),
                                                   1**2 * WhiteKernel(noise_level=1)]})

In [84]:
clf.best_params_

{'kernel': 1**2 * RationalQuadratic(alpha=1, length_scale=1)}

In [85]:
model = GaussianProcessClassifier(**clf.best_params_, max_iter_predict = 1000)

model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9106753812636166


In [88]:
# Gradient Boosting Classifier

model = GradientBoostingClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.9019607843137255

In [89]:
# Optimizing the Gradient Boosting Classifier

model = GradientBoostingClassifier()

parameters = {
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10,100]
}

clf  = BayesSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=10, estimator=GradientBoostingClassifier(),
              search_spaces={'learning_rate': [0.01, 0.1, 1, 10, 100],
                             'max_depth': [1, 3, 5, 7, 9],
                             'n_estimators': [5, 50, 250, 500]})

In [90]:
clf.best_params_

OrderedDict([('learning_rate', 0.1), ('max_depth', 9), ('n_estimators', 500)])

In [91]:
model = GradientBoostingClassifier(**clf.best_params_)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9259259259259259


In [92]:
# Ada Boost Classifier

model = AdaBoostClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.8714596949891068

In [93]:
# Optimizing the Gradient Boosting Classifier

model = AdaBoostClassifier()

parameters = {
    "n_estimators":[5,50,250,500],
    "learning_rate":[0.01,0.1,1,10,100]
}

clf  = RandomizedSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:561: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight *
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:561: RuntimeWarning: overflow encountered in exp
  sample_weight *= np.exp(estimator_weight *
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:155: RuntimeWarning: invalid value encountered in true_divide
  sample_weight /= sample_weight_sum
/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py", line 443, in fit
    return super().fit(X, y, sample_weight)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan 0.143314 0.143314 0.863893 0.871745 nan 0.143314 0.870438 0.862590
 0.853414]
  warnings.warn(


RandomizedSearchCV(cv=10, estimator=AdaBoostClassifier(),
                   param_distributions={'learning_rate': [0.01, 0.1, 1, 10,
                                                          100],
                                        'n_estimators': [5, 50, 250, 500]})

In [94]:
clf.best_params_

{'n_estimators': 500, 'learning_rate': 0.01}

In [95]:
model = AdaBoostClassifier(**clf.best_params_)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9041394335511983


In [ ]:
#pickle.dump(model, open("ada_boost_model.dat", "wb"))


In [96]:
# Extra trees classifier

model = ExtraTreesClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.9084967320261438

In [97]:
# Optimizing the SGDClassifier

model = ExtraTreesClassifier()

parameters = {
    'n_estimators': list(range(50,126,25)),
    'min_samples_leaf': list(range(1,20,1)),
    'min_samples_split': list(range(1,20,1))
}

clf  = RandomizedSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/joblib/pa

/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/cconsta1/opt/anaconda3/lib/python3.9/site-packages/joblib/pa

RandomizedSearchCV(cv=10, estimator=ExtraTreesClassifier(),
                   param_distributions={'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15, 16,
                                                             17, 18, 19],
                                        'min_samples_split': [1, 2, 3, 4, 5, 6,
                                                              7, 8, 9, 10, 11,
                                                              12, 13, 14, 15,
                                                              16, 17, 18, 19],
                                        'n_estimators': [50, 75, 100, 125]})

In [98]:
clf.best_params_

{'n_estimators': 125, 'min_samples_split': 4, 'min_samples_leaf': 4}

In [99]:
model = ExtraTreesClassifier(**clf.best_params_)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9128540305010894


In [101]:
# Light boosting regressor

model = lgb.LGBMClassifier()

model.fit(X_train, y_train)

model.score(X_test, y_test)

0.8888888888888888

In [115]:
# Optimizing the SGDClassifier

model = lgb.LGBMClassifier()

parameters = {
    'num_leaves': [5, 10, 20, 31, 50, 100, 125, 150, 200], 
    'min_child_samples': [20, 30, 50 , 100, 200, 300, 400], 
    'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 10, 100],
    'reg_alpha': [0, 1e-1, 1, 2, 3, 5, 10],
    'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 40, 50]
}

clf  = RandomizedSearchCV(model, parameters, cv=10, return_train_score=False)

clf.fit(X,y)

RandomizedSearchCV(cv=10, estimator=LGBMClassifier(),
                   param_distributions={'min_child_samples': [20, 30, 50, 100,
                                                              200, 300, 400],
                                        'min_child_weight': [1e-05, 0.001, 0.01,
                                                             0.1, 1, 10, 100],
                                        'num_leaves': [5, 10, 20, 31, 50, 100,
                                                       125, 150, 200],
                                        'reg_alpha': [0, 0.1, 1, 2, 3, 5, 10],
                                        'reg_lambda': [0, 0.1, 1, 5, 10, 20, 40,
                                                       50]})

In [116]:
clf.best_params_

{'reg_lambda': 0,
 'reg_alpha': 0,
 'num_leaves': 125,
 'min_child_weight': 0.1,
 'min_child_samples': 300}

In [117]:
model = lgb.LGBMClassifier(**clf.best_params_)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.9128540305010894


In [118]:
pickle.dump(model, open("lgb_model_20220522.dat", "wb"))


In [185]:
# Missing data models

count_low_percentage = 0
no_models = 0

for k in range(1,12):
    for combo in combinations(cols, k):  # 2 for pairs, 3 for triplets, etc
        # take all combinations of certain length
        col_list = list(combo)
        # append the Sex column
        col_list.append("Sex")
    
        #Create DataFrames with the selected columns
        df = iter_imputed_data_goldman.filter(col_list)
    
        # Shuffle the dat
        df = df.sample(frac=1).reset_index(drop=True)
    
        # Create feature matrix and target vector
        X = df.drop('Sex', axis = 1).values
        y = df['Sex']
        
        #print(len(y[y==1]))
        
        #split the dataset
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size=0.3, stratify=y)
        
        clf =  LinearDiscriminantAnalysis()
        #clf  = GridSearchCV(classifier, parameters, cv=20)
    
        #with parallel_backend('threading', n_jobs=-1):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        
        
        if(score < 0.85):
            count_low_percentage = count_low_percentage + 1
            
        no_models = no_models+1
            
            
            
        filename = "model_lda_goldman_"+"_".join(combo)+"_.dat"
         
        accuracy_filename = "model_lda_goldman_"+"_".join(combo)+"_.txt"
        
        with open(accuracy_filename, "w") as text_file:
            text_file.write(str(score))
                 
        print(filename)
        print(score)
        
        classifier = LinearDiscriminantAnalysis()
        classifier.fit(X,y)
        
        pickle.dump(classifier, open(filename, "wb"))

print(count_low_percentage)
print(no_models)

model_lda_goldman_BIB_.dat
0.6579520697167756
model_lda_goldman_HML_.dat
0.7668845315904139
model_lda_goldman_HHD_.dat
0.8605664488017429
model_lda_goldman_RML_.dat
0.7821350762527233
model_lda_goldman_FML_.dat
0.775599128540305
model_lda_goldman_FBL_.dat
0.7581699346405228
model_lda_goldman_FHD_.dat
0.7864923747276689
model_lda_goldman_TML_.dat
0.7276688453159041
model_lda_goldman_FEB_.dat
0.8518518518518519
model_lda_goldman_TPB_.dat
0.8518518518518519
model_lda_goldman_HEB_.dat
0.8431372549019608
model_lda_goldman_BIB_HML_.dat
0.7559912854030502
model_lda_goldman_BIB_HHD_.dat
0.8714596949891068
model_lda_goldman_BIB_RML_.dat
0.7864923747276689
model_lda_goldman_BIB_FML_.dat
0.7647058823529411
model_lda_goldman_BIB_FBL_.dat
0.775599128540305
model_lda_goldman_BIB_FHD_.dat
0.8344226579520697
model_lda_goldman_BIB_TML_.dat
0.7494553376906318
model_lda_goldman_BIB_FEB_.dat
0.8431372549019608
model_lda_goldman_BIB_TPB_.dat
0.8431372549019608
model_lda_goldman_BIB_HEB_.dat
0.8409586056644

model_lda_goldman_HHD_TML_FEB_.dat
0.8758169934640523
model_lda_goldman_HHD_TML_TPB_.dat
0.8562091503267973
model_lda_goldman_HHD_TML_HEB_.dat
0.8823529411764706
model_lda_goldman_HHD_FEB_TPB_.dat
0.8736383442265795
model_lda_goldman_HHD_FEB_HEB_.dat
0.8627450980392157
model_lda_goldman_HHD_TPB_HEB_.dat
0.8736383442265795
model_lda_goldman_RML_FML_FBL_.dat
0.8082788671023965
model_lda_goldman_RML_FML_FHD_.dat
0.8888888888888888
model_lda_goldman_RML_FML_TML_.dat
0.7777777777777778
model_lda_goldman_RML_FML_FEB_.dat
0.8714596949891068
model_lda_goldman_RML_FML_TPB_.dat
0.8453159041394336
model_lda_goldman_RML_FML_HEB_.dat
0.8322440087145969
model_lda_goldman_RML_FBL_FHD_.dat
0.8910675381263616
model_lda_goldman_RML_FBL_TML_.dat
0.7952069716775599
model_lda_goldman_RML_FBL_FEB_.dat
0.8976034858387799
model_lda_goldman_RML_FBL_TPB_.dat
0.8605664488017429
model_lda_goldman_RML_FBL_HEB_.dat
0.8191721132897604
model_lda_goldman_RML_FHD_TML_.dat
0.8474945533769063
model_lda_goldman_RML_FHD_FE

model_lda_goldman_BIB_RML_TPB_HEB_.dat
0.8736383442265795
model_lda_goldman_BIB_FML_FBL_FHD_.dat
0.8453159041394336
model_lda_goldman_BIB_FML_FBL_TML_.dat
0.738562091503268
model_lda_goldman_BIB_FML_FBL_FEB_.dat
0.8496732026143791
model_lda_goldman_BIB_FML_FBL_TPB_.dat
0.8518518518518519
model_lda_goldman_BIB_FML_FBL_HEB_.dat
0.8431372549019608
model_lda_goldman_BIB_FML_FHD_TML_.dat
0.8496732026143791
model_lda_goldman_BIB_FML_FHD_FEB_.dat
0.8649237472766884
model_lda_goldman_BIB_FML_FHD_TPB_.dat
0.8627450980392157
model_lda_goldman_BIB_FML_FHD_HEB_.dat
0.8779956427015251
model_lda_goldman_BIB_FML_TML_FEB_.dat
0.8801742919389978
model_lda_goldman_BIB_FML_TML_TPB_.dat
0.869281045751634
model_lda_goldman_BIB_FML_TML_HEB_.dat
0.8344226579520697
model_lda_goldman_BIB_FML_FEB_TPB_.dat
0.8583877995642701
model_lda_goldman_BIB_FML_FEB_HEB_.dat
0.8649237472766884
model_lda_goldman_BIB_FML_TPB_HEB_.dat
0.8779956427015251
model_lda_goldman_BIB_FBL_FHD_TML_.dat
0.8453159041394336
model_lda_goldma

model_lda_goldman_HHD_FML_TPB_HEB_.dat
0.8518518518518519
model_lda_goldman_HHD_FBL_FHD_TML_.dat
0.8714596949891068
model_lda_goldman_HHD_FBL_FHD_FEB_.dat
0.8453159041394336
model_lda_goldman_HHD_FBL_FHD_TPB_.dat
0.8322440087145969
model_lda_goldman_HHD_FBL_FHD_HEB_.dat
0.8496732026143791
model_lda_goldman_HHD_FBL_TML_FEB_.dat
0.8714596949891068
model_lda_goldman_HHD_FBL_TML_TPB_.dat
0.8605664488017429
model_lda_goldman_HHD_FBL_TML_HEB_.dat
0.8736383442265795
model_lda_goldman_HHD_FBL_FEB_TPB_.dat
0.8758169934640523
model_lda_goldman_HHD_FBL_FEB_HEB_.dat
0.8779956427015251
model_lda_goldman_HHD_FBL_TPB_HEB_.dat
0.8649237472766884
model_lda_goldman_HHD_FHD_TML_FEB_.dat
0.8671023965141612
model_lda_goldman_HHD_FHD_TML_TPB_.dat
0.8562091503267973
model_lda_goldman_HHD_FHD_TML_HEB_.dat
0.8453159041394336
model_lda_goldman_HHD_FHD_FEB_TPB_.dat
0.8758169934640523
model_lda_goldman_HHD_FHD_FEB_HEB_.dat
0.8605664488017429
model_lda_goldman_HHD_FHD_TPB_HEB_.dat
0.8496732026143791
model_lda_gold

model_lda_goldman_BIB_HML_FML_FBL_TPB_.dat
0.8714596949891068
model_lda_goldman_BIB_HML_FML_FBL_HEB_.dat
0.8431372549019608
model_lda_goldman_BIB_HML_FML_FHD_TML_.dat
0.8910675381263616
model_lda_goldman_BIB_HML_FML_FHD_FEB_.dat
0.8649237472766884
model_lda_goldman_BIB_HML_FML_FHD_TPB_.dat
0.8518518518518519
model_lda_goldman_BIB_HML_FML_FHD_HEB_.dat
0.8823529411764706
model_lda_goldman_BIB_HML_FML_TML_FEB_.dat
0.8627450980392157
model_lda_goldman_BIB_HML_FML_TML_TPB_.dat
0.8344226579520697
model_lda_goldman_BIB_HML_FML_TML_HEB_.dat
0.8496732026143791
model_lda_goldman_BIB_HML_FML_FEB_TPB_.dat
0.8562091503267973
model_lda_goldman_BIB_HML_FML_FEB_HEB_.dat
0.8627450980392157
model_lda_goldman_BIB_HML_FML_TPB_HEB_.dat
0.8387799564270153
model_lda_goldman_BIB_HML_FBL_FHD_TML_.dat
0.840958605664488
model_lda_goldman_BIB_HML_FBL_FHD_FEB_.dat
0.8605664488017429
model_lda_goldman_BIB_HML_FBL_FHD_TPB_.dat
0.8888888888888888
model_lda_goldman_BIB_HML_FBL_FHD_HEB_.dat
0.8888888888888888
model_lda

...

model_lda_goldman_BIB_HML_HHD_FHD_TML_TPB_HEB_.dat
0.8671023965141612
model_lda_goldman_BIB_HML_HHD_FHD_FEB_TPB_HEB_.dat
0.8997821350762527
model_lda_goldman_BIB_HML_HHD_TML_FEB_TPB_HEB_.dat
0.8845315904139434
model_lda_goldman_BIB_HML_RML_FML_FBL_FHD_TML_.dat
0.8605664488017429
model_lda_goldman_BIB_HML_RML_FML_FBL_FHD_FEB_.dat
0.869281045751634
model_lda_goldman_BIB_HML_RML_FML_FBL_FHD_TPB_.dat
0.8736383442265795
model_lda_goldman_BIB_HML_RML_FML_FBL_FHD_HEB_.dat
0.8649237472766884
model_lda_goldman_BIB_HML_RML_FML_FBL_TML_FEB_.dat
0.8845315904139434
model_lda_goldman_BIB_HML_RML_FML_FBL_TML_TPB_.dat
0.8649237472766884
model_lda_goldman_BIB_HML_RML_FML_FBL_TML_HEB_.dat
0.8344226579520697
model_lda_goldman_BIB_HML_RML_FML_FBL_FEB_TPB_.dat
0.8736383442265795
model_lda_goldman_BIB_HML_RML_FML_FBL_FEB_HEB_.dat
0.8823529411764706
model_lda_goldman_BIB_HML_RML_FML_FBL_TPB_HEB_.dat
0.8431372549019608
model_lda_goldman_BIB_HML_RML_FML_FHD_TML_FEB_.dat
0.869281045751634
model_lda_goldman_BIB_

model_lda_goldman_BIB_RML_FML_FHD_TML_FEB_TPB_.dat
0.8801742919389978
model_lda_goldman_BIB_RML_FML_FHD_TML_FEB_HEB_.dat
0.8758169934640523
model_lda_goldman_BIB_RML_FML_FHD_TML_TPB_HEB_.dat
0.8801742919389978
model_lda_goldman_BIB_RML_FML_FHD_FEB_TPB_HEB_.dat
0.8867102396514162
model_lda_goldman_BIB_RML_FML_TML_FEB_TPB_HEB_.dat
0.8627450980392157
model_lda_goldman_BIB_RML_FBL_FHD_TML_FEB_TPB_.dat
0.8932461873638344
model_lda_goldman_BIB_RML_FBL_FHD_TML_FEB_HEB_.dat
0.8888888888888888
model_lda_goldman_BIB_RML_FBL_FHD_TML_TPB_HEB_.dat
0.8867102396514162
model_lda_goldman_BIB_RML_FBL_FHD_FEB_TPB_HEB_.dat
0.8801742919389978
model_lda_goldman_BIB_RML_FBL_TML_FEB_TPB_HEB_.dat
0.8801742919389978
model_lda_goldman_BIB_RML_FHD_TML_FEB_TPB_HEB_.dat
0.8649237472766884
model_lda_goldman_BIB_FML_FBL_FHD_TML_FEB_TPB_.dat
0.8736383442265795
model_lda_goldman_BIB_FML_FBL_FHD_TML_FEB_HEB_.dat
0.8779956427015251
model_lda_goldman_BIB_FML_FBL_FHD_TML_TPB_HEB_.dat
0.8867102396514162
model_lda_goldman_BI

model_lda_goldman_HHD_FML_FBL_FHD_TML_FEB_HEB_.dat
0.8540305010893247
model_lda_goldman_HHD_FML_FBL_FHD_TML_TPB_HEB_.dat
0.8496732026143791
model_lda_goldman_HHD_FML_FBL_FHD_FEB_TPB_HEB_.dat
0.8671023965141612
model_lda_goldman_HHD_FML_FBL_TML_FEB_TPB_HEB_.dat
0.8823529411764706
model_lda_goldman_HHD_FML_FHD_TML_FEB_TPB_HEB_.dat
0.8474945533769063
model_lda_goldman_HHD_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8976034858387799
model_lda_goldman_RML_FML_FBL_FHD_TML_FEB_TPB_.dat
0.8671023965141612
model_lda_goldman_RML_FML_FBL_FHD_TML_FEB_HEB_.dat
0.8649237472766884
model_lda_goldman_RML_FML_FBL_FHD_TML_TPB_HEB_.dat
0.8627450980392157
model_lda_goldman_RML_FML_FBL_FHD_FEB_TPB_HEB_.dat
0.8649237472766884
model_lda_goldman_RML_FML_FBL_TML_FEB_TPB_HEB_.dat
0.8649237472766884
model_lda_goldman_RML_FML_FHD_TML_FEB_TPB_HEB_.dat
0.8779956427015251
model_lda_goldman_RML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8779956427015251
model_lda_goldman_FML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8845315904139434
model_lda_goldman_BI

model_lda_goldman_BIB_HHD_FML_FHD_TML_FEB_TPB_HEB_.dat
0.8758169934640523
model_lda_goldman_BIB_HHD_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8845315904139434
model_lda_goldman_BIB_RML_FML_FBL_FHD_TML_FEB_TPB_.dat
0.8736383442265795
model_lda_goldman_BIB_RML_FML_FBL_FHD_TML_FEB_HEB_.dat
0.8671023965141612
model_lda_goldman_BIB_RML_FML_FBL_FHD_TML_TPB_HEB_.dat
0.8888888888888888
model_lda_goldman_BIB_RML_FML_FBL_FHD_FEB_TPB_HEB_.dat
0.8954248366013072
model_lda_goldman_BIB_RML_FML_FBL_TML_FEB_TPB_HEB_.dat
0.8714596949891068
model_lda_goldman_BIB_RML_FML_FHD_TML_FEB_TPB_HEB_.dat
0.9019607843137255
model_lda_goldman_BIB_RML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8605664488017429
model_lda_goldman_BIB_FML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8823529411764706
model_lda_goldman_HML_HHD_RML_FML_FBL_FHD_TML_FEB_.dat
0.8823529411764706
model_lda_goldman_HML_HHD_RML_FML_FBL_FHD_TML_TPB_.dat
0.8562091503267973
model_lda_goldman_HML_HHD_RML_FML_FBL_FHD_TML_HEB_.dat
0.8627450980392157
model_lda_goldman_HML_HHD_RML_FML_FBL_

model_lda_goldman_BIB_HML_HHD_RML_FML_FBL_TML_FEB_TPB_HEB_.dat
0.8932461873638344
model_lda_goldman_BIB_HML_HHD_RML_FML_FHD_TML_FEB_TPB_HEB_.dat
0.8736383442265795
model_lda_goldman_BIB_HML_HHD_RML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8801742919389978
model_lda_goldman_BIB_HML_HHD_FML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8671023965141612
model_lda_goldman_BIB_HML_RML_FML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8823529411764706
model_lda_goldman_BIB_HHD_RML_FML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8997821350762527
model_lda_goldman_HML_HHD_RML_FML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8823529411764706
model_lda_goldman_BIB_HML_HHD_RML_FML_FBL_FHD_TML_FEB_TPB_HEB_.dat
0.8910675381263616
326
2047
